In [2]:
import numpy as np     
import pandas as pd              
import matplotlib.pyplot as plt
from matplotlib import cm

# Clases propias
from SignalClases import OrgSignal
from SignalClases import FiltSignal
from SignalClases import SWSignal
from SignalClases import IndSW
from SignalClases import saveData
from ImgPlot import Plot_Signals
from Hilbert import HilbClass
from CrossDetClass import crossDetector
from timeDetClass import tDetClass
from recSigClass import recClass
from saveList import save

In [3]:
## Inicio de algoritmo 

# Indices para carpetas a revisar 
# numFol = 4
# numFile = [10,4,9,8]
# numLabel = 8
# srate = 10000                               

# # Creación de nuevos 4 registros 
# numFol = 4
# numFile = [3,2,2,3]
# numLabel = 8
# srate = 10000    

# Creación de ultimos 4 registros 
numFol = 4
numFile = [0,3,6,7] # 77 CBX, 91 NaCl, 94 QUININA, 50 TMA
numLabel = 8
srate = 10000                              
        
        

In [6]:
for a in range(0,numFol):                       # (0,numFol)
    Foldnum = a
    Filenum = numFile[a]
    # Extracción de datos de las clases
    orgSig,path, fileNm = OrgSignal.getData(Foldnum,Filenum)              # Señal original
    szOrg = orgSig.shape
    print(type(orgSig))
    print(szOrg)

    filtSig,filtPath = FiltSignal.getData(Foldnum,Filenum)           # Señal Filtrada
    print(type(orgSig))
    print(filtSig.shape)

    SWSig = SWSignal.getData(Foldnum,Filenum)               # Señal filtrada para detección de ondas lentas 
    print(type(orgSig))
    print(filtSig.shape)

    # Variables de interés
    timevec = np.arange(0,szOrg[1],1)/srate    
    ## Variables necesarias 
    
    
    for c in range(0,numLabel):                # (0,numLabel)
        lst = []
        label = c        
        indSW,labelInd = IndSW.getData(Foldnum,Filenum,label)            # Indices de tiempo de detección de onda lenta  (0 ->HAD1)
        print(labelInd)
        g = indSW.shape
        print(type(g))          
        print(g)                                                # Indices de FR en algoritmo     
        
        for d in range(0,g[0]):                                 # Recorrer indices de ondas lentas identificadas range(0,g[0])
            BPSig = filtSig[label,indSW[d,0]:indSW[d,1]].copy()
            
            try:
                ampEnv,tHold,medT = HilbClass.HBSig(BPSig)
            except:
                continue
            
            preCross, postCross = crossDetector.crossDet(ampEnv,tHold)   # Detección de cruce inicial por threshold 
            if not preCross or not postCross: continue
            
            starIndSR,endIndSR = crossDetector.detDur(preCross,postCross,ampEnv,medT)  # Detección original del cruce por 0.5*threshold los indices están relacionados con la señal cortada
            if not starIndSR or not endIndSR: continue
            
            indStar6MS,indEnd6MS,lapse6 = tDetClass.timeLen(starIndSR,endIndSR,timevec[indSW[d,0]:indSW[d,1]]) # Indices de inicio y fin de tiempo relacionado con extracto de signal (no son coordenadas de signal original) 
            if not indStar6MS or not indEnd6MS: continue 
            
            indStar, indEnd, lapse = tDetClass.time10Len(indStar6MS,indEnd6MS,timevec[indSW[d,0]:indSW[d,1]])
            if not indStar or not indEnd: continue
            
            recSig, recTr = recClass.recClassThre(BPSig) # Señal rectificada del fragmeto de onda lenta
            
            trueStart,trueEnd = crossDetector.finCross(indStar, recSig,indEnd,recTr,indSW,d)  # última comprobacion de señal, indices relacionados con señal de tiempo orginal
            
            if not trueStart or not trueEnd: continue
                
            lst = save.saveList(lst, d, trueStart,fileNm,label,labelInd,indSW,trueEnd,lapse)
        
        df1 = pd.DataFrame(lst, columns=['File Name','Electrode Name','Electrode num','indSW I','indSW F','indFR I','indFR F','Duration'])
        pathSV = saveData.getData(Foldnum,Filenum,label)
        df1.to_csv(pathSV,sep=',',index=False)     
             

D:\\Base de datos proyecto sin repeticion\\experimental CBX proyecto\\FR-84_CBX_n.mat
<class 'numpy.ndarray'>
(8, 16660001)
D:\\Base de datos proyecto sin repeticion\\experimental NaCl proyecto\\FR-89_NaCl_n.mat
<class 'numpy.ndarray'>
(8, 17610001)
D:\\Base de datos proyecto sin repeticion\\experimental QUININA proyecto\\FR-79_QUININA_n.mat
<class 'numpy.ndarray'>
(8, 17220001)
D:\\Base de datos proyecto sin repeticion\\experimental TMA proyecto\\FR-91_TMA_n.mat
<class 'numpy.ndarray'>
(8, 17710001)


In [5]:
for a in range(0,numFol):                       # (0,numFol)
    for b in range(0,numFile[a]):               # (0,numFile[a])  
        Foldnum = a
        Filenum = b
        # Extracción de datos de las clases
        orgSig,path, fileNm = OrgSignal.getData(Foldnum,Filenum)              # Señal original
        szOrg = orgSig.shape
        print(type(orgSig))
        print(szOrg)

        filtSig,filtPath = FiltSignal.getData(Foldnum,Filenum)           # Señal Filtrada
        print(type(orgSig))
        print(filtSig.shape)

        SWSig = SWSignal.getData(Foldnum,Filenum)               # Señal filtrada para detección de ondas lentas 
        print(type(orgSig))
        print(filtSig.shape)

        # Variables de interés
        timevec = np.arange(0,szOrg[1],1)/srate    
        ## Variables necesarias 
        
        
        for c in range(0,numLabel):                # (0,numLabel)
            lst = []
            label = c        
            indSW,labelInd = IndSW.getData(Foldnum,Filenum,label)            # Indices de tiempo de detección de onda lenta  (0 ->HAD1)
            print(labelInd)
            g = indSW.shape
            print(type(g))          
            print(g)                                                # Indices de FR en algoritmo     
            
            for d in range(0,g[0]):                                 # Recorrer indices de ondas lentas identificadas range(0,g[0])
                BPSig = filtSig[label,indSW[d,0]:indSW[d,1]].copy()
                
                try:
                    ampEnv,tHold,medT = HilbClass.HBSig(BPSig)
                except:
                    continue
                
                preCross, postCross = crossDetector.crossDet(ampEnv,tHold)   # Detección de cruce inicial por threshold 
                if not preCross or not postCross: continue
                
                starIndSR,endIndSR = crossDetector.detDur(preCross,postCross,ampEnv,medT)  # Detección original del cruce por 0.5*threshold los indices están relacionados con la señal cortada
                if not starIndSR or not endIndSR: continue
                
                indStar6MS,indEnd6MS,lapse6 = tDetClass.timeLen(starIndSR,endIndSR,timevec[indSW[d,0]:indSW[d,1]]) # Indices de inicio y fin de tiempo relacionado con extracto de signal (no son coordenadas de signal original) 
                if not indStar6MS or not indEnd6MS: continue 
                
                indStar, indEnd, lapse = tDetClass.time10Len(indStar6MS,indEnd6MS,timevec[indSW[d,0]:indSW[d,1]])
                if not indStar or not indEnd: continue
                
                recSig, recTr = recClass.recClassThre(BPSig) # Señal rectificada del fragmeto de onda lenta
                
                trueStart,trueEnd = crossDetector.finCross(indStar, recSig,indEnd,recTr,indSW,d)  # última comprobacion de señal, indices relacionados con señal de tiempo orginal
                
                if not trueStart or not trueEnd: continue
                    
                lst = save.saveList(lst, d, trueStart,fileNm,label,labelInd,indSW,trueEnd,lapse)
            
            df1 = pd.DataFrame(lst, columns=['File Name','Electrode Name','Electrode num','indSW I','indSW F','indFR I','indFR F','Duration'])
            pathSV = saveData.getData(Foldnum,Filenum,label)
            df1.to_csv(pathSV,sep=',',index=False)     
             

D:\\Base de datos proyecto sin repeticion\\experimental CBX proyecto\\FR-77_CBX_n.mat
<class 'numpy.ndarray'>
(8, 17100001)
D:\\Base de datos proyecto sin repeticion\\experimental CBX proyecto\\FR-78_CBX_n.mat
<class 'numpy.ndarray'>
(8, 17100001)
D:\\Base de datos proyecto sin repeticion\\experimental CBX proyecto\\FR-80_CBX_n.mat
<class 'numpy.ndarray'>
(8, 15880001)
D:\\Base de datos proyecto sin repeticion\\experimental NaCl proyecto\\R-45_NaCl_n.mat


KeyboardInterrupt: 

In [ ]:

                    
for c in range(0,1):                # (0,numLabel)
    lst = []
    label = c        
    indSW,labelInd = IndSW.getData(Foldnum,Filenum,label)            # Indices de tiempo de detección de onda lenta  (0 ->HAD1)
    print(labelInd)
    g = indSW.shape
    print(type(g))          
    print(g)                                                # Indices de FR en algoritmo     
     
    for d in range(0,20):                                 # Recorrer indices de ondas lentas identificadas range(0,g[0])
        BPSig = filtSig[label,indSW[d,0]:indSW[d,1]].copy()
        
        ampEnv,tHold,medT = HilbClass.HBSig(BPSig) 
        
        preCross, postCross = crossDetector.crossDet(ampEnv,tHold)   # Detección de cruce inicial por threshold 
        if not preCross or not postCross: continue
        
        starIndSR,endIndSR = crossDetector.detDur(preCross,postCross,ampEnv,medT)  # Detección original del cruce por 0.5*threshold los indices están relacionados con la señal cortada
        if not starIndSR or not endIndSR: continue
        
        indStar6MS,indEnd6MS,lapse6 = tDetClass.timeLen(starIndSR,endIndSR,timevec[indSW[d,0]:indSW[d,1]]) # Indices de inicio y fin de tiempo relacionado con extracto de signal (no son coordenadas de signal original) 
        if not indStar6MS or not indEnd6MS: continue 
        
        indStar, indEnd, lapse = tDetClass.time10Len(indStar6MS,indEnd6MS,timevec[indSW[d,0]:indSW[d,1]])
        if not indStar or not indEnd: continue
        
        recSig, recTr = recClass.recClassThre(BPSig) # Señal rectificada del fragmeto de onda lenta
        
        trueStart,trueEnd = crossDetector.finCross(indStar, recSig,indEnd,recTr,indSW,d)  # última comprobacion de señal, indices relacionados con señal de tiempo orginal
        
        if not trueStart or not trueEnd: continue
        
        # for ind in range(len(trueStart)):
        #     listTemp = [fileNm, [labelInd,label], [indSW[d,0],indSW[d,1]],path,filtPath,[trueStart[ind],trueEnd[ind]]]
        #     lst.append(listTemp)
            
        lst = save.saveList(lst, d, trueStart,fileNm,label,labelInd,indSW,trueEnd,lapse)
    
            
    df1 = pd.DataFrame(lst, columns=['File Name','Electrode','indSW','indFR','Duration'])
    pathSV = saveData.getData(Foldnum,Filenum,label)
    df1.to_csv(pathSV,sep=',',index=False)                    

## Pruebas

### Pruebas (separamiento en un solo intento )

In [ ]:
import numpy as np     
import pandas as pd              
import matplotlib.pyplot as plt
from matplotlib import cm

# Clases propias
from SignalClases import OrgSignal
from SignalClases import FiltSignal
from SignalClases import SWSignal
from SignalClases import IndSW
from ImgPlot import Plot_Signals
from Hilbert import HilbClass
from CrossDetClass import crossDetector
from timeDetClass import tDetClass
from recSigClass import recClass
from saveList import save

In [ ]:
numFol = 4
numFile = [10,4,9,8]
numLabel = 8

Foldnum = 0
Filenum = 8

# Extracción de datos de las clases
orgSig,path, fileNm = OrgSignal.getData(Foldnum,Filenum)  # Señal original
szOrg = orgSig.shape
print(type(orgSig))
print(orgSig.shape)

filtSig,filtPath = FiltSignal.getData(Foldnum,Filenum)           # Señal Filtrada
print(type(filtSig))
print(filtSig.shape)

SWSig = SWSignal.getData(Foldnum,Filenum)               # Señal filtrada para detección de ondas lentas 
print(type(SWSig))
print(SWSig.shape)

In [ ]:
label = 7

indSW,labelInd = IndSW.getData(Foldnum,Filenum,label)            # Indices de tiempo de detección de onda lenta  (0 ->HAD1)
g = indSW.shape
print(type(g))          
print(g)                                                # Indices de FR en algoritmo                               

# Variables de interés
srate = 10000                               
timevec = np.arange(0,szOrg[1],1)/srate    
## Variables necesaria

In [ ]:
d = 0
numInd = 0

# Grafica señal original
fig, axs = plt.subplots(3, sharex=True)
axs[0].plot(timevec[indSW[numInd,0]:indSW[numInd,1]],orgSig[label,indSW[numInd,0]:indSW[numInd,1]])    # Señal Orginal
axs[0].set_title('Original Signal')
axs[0].set_title('95_CBX', loc='right', style = "italic")
axs[1].plot(timevec[indSW[numInd,0]:indSW[numInd,1]],SWSig[label,indSW[numInd,0]:indSW[numInd,1]])  # Senal Slow Wave
axs[1].set_title('Slow Wave filtering Signal')
axs[2].plot(timevec[indSW[numInd,0]:indSW[numInd,1]],filtSig[label,indSW[numInd,0]:indSW[numInd,1]]*.001)   # Señal band pass 250-500 Hz
axs[2].set_title('Band Pass Signal (250 -500 Hz)')
fig.text(0.5, 0.04, 'Time (s)', ha='center', va='center')
fig.text(0.06, 0.5, 'Amplitude (V)', ha='center', va='center', rotation='vertical')

In [ ]:
d = 0
numInd = 0
print(indSW[numInd,0])
print(indSW[numInd,1])

In [ ]:
srate = 10000                               
timevec = np.arange(0,szOrg[1],1)/srate    
BPSig = filtSig[label,indSW[d,0]:indSW[d,1]].copy()

ampEnv,tHold,medT = HilbClass.HBSig(BPSig)

In [ ]:
plt.plot(timevec[indSW[numInd,0]:indSW[numInd,1]],BPSig,'b', label='Bandpass] Signal')   # Plot de señal y tiempo en el que puede estar pasando FR (señal recortada)
plt.plot(timevec[indSW[numInd,0]:indSW[numInd,1]],ampEnv[0:],'r--',label='Hilbert Envelope')
plt.axhline(y=tHold, color='g', linestyle='-.', label='Threshold')
plt.axhline(y=medT, color='y', linestyle='-.', label='0.5 * Threshold')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (V)")
plt.title('Envelope of filter signal')
plt.title('95_CBX', loc='right', style = "italic")
plt.legend()
plt.xlim([2.5,2.7])

In [ ]:
#Plot_Signals.Plot_Thold(timevec, indSW, d, BPSig, ampEnv,tHold,medT)
timevec = np.arange(0,szOrg[1],1)/srate    
preCross, postCross = crossDetector.crossDet(ampEnv,tHold)   # Detección de cruce inicial por threshold 
print(preCross,postCross)
preCrossCopy = preCross.copy()
postCrossCopy = postCross.copy()


starIndSR,endIndSR = crossDetector.detDur(preCrossCopy,postCrossCopy,ampEnv,medT)  # Detección original del cruce por 0.5*threshold los indices están relacionados con la señal cortada
print(starIndSR,endIndSR)
# print(preCross,postCross)
starIndSRCopy = starIndSR.copy()
endIndSRCopy = endIndSR.copy()


In [ ]:
czind = 1

timeCut = timevec[indSW[numInd,0]:indSW[numInd,1]]

plt.plot(timeCut[starIndSR[czind]:endIndSR[czind]],BPSig[starIndSR[czind]:endIndSR[czind]],'b', label='Bandpass] Signal')   # Plot de señal y tiempo en el que puede estar pasando FR
plt.plot(timeCut[starIndSR[czind]:endIndSR[czind]],ampEnv[starIndSR[czind]:endIndSR[czind]],'r--',label='Hilbert Envelope')   # Envoltura de la señal
plt.plot(timeCut[starIndSR[czind]],ampEnv[starIndSR[czind]],'c*',markersize=10, label = 'Event duration')                        # Punto inicial cruce FR
plt.plot(timeCut[endIndSR[czind]],ampEnv[endIndSR[czind]],'c*',markersize=8)
# plt.plot(timeCut[preCross[0]],ampEnv[preCross[1]],'ms',markersize=8, label ='Event marked')                        # Punto inicial cruce FR
# plt.plot(timeCut[postCross[0]],ampEnv[postCross[1]],'ms',markersize=8)
plt.plot(timeCut[preCross[1]],ampEnv[preCross[1]],'ms',markersize=8, label ='Event marked')                        # Punto inicial cruce FR
plt.plot(timeCut[postCross[1]],ampEnv[postCross[1]],'ms',markersize=8)
plt.plot(timeCut[preCross[2]],ampEnv[preCross[2]],'mo',markersize=8, label ='Event marked')                        # Punto inicial cruce FR
plt.plot(timeCut[postCross[2]],ampEnv[postCross[2]],'mo',markersize=8)
plt.axhline(y=tHold, color='g', linestyle='-.', label='Threshold')
plt.axhline(y=medT, color='y', linestyle='-.', label='0.5 * Threshold')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (V)")
plt.title('Envelope of filter signal')
plt.legend(prop = { "size": 8 } , loc ="lower right")

In [ ]:
indStar6MS,indEnd6MS,lapse6 = tDetClass.timeLen(starIndSRCopy,endIndSRCopy,timevec[indSW[d,0]:indSW[d,1]]) # Indices de inicio y fin de tiempo relacionado con extracto de signal (no son coordenadas de signal original) 
indStar6MSCopy = indStar6MS.copy()
indEnd6MSCopy = indEnd6MS.copy()
print(indStar6MS,indEnd6MS,lapse6)

In [ ]:
indStar, indEnd, lapse = tDetClass.time10Len(indStar6MSCopy,indEnd6MSCopy,timevec[indSW[d,0]:indSW[d,1]])
print(indStar,indEnd,lapse)
indStarCopy = indStar.copy()
indEndCopy = indEnd.copy()


In [ ]:
recSig, recTr = recClass.recClassThre(BPSig) # Señal rectificada del fragmeto de onda lenta
trueStart,trueEnd = crossDetector.finCross(indStarCopy, recSig,indEndCopy,recTr,indSW,d)  # última comprobacion de señal, indices relacionados con señal de tiempo orginal
print(trueStart,trueEnd)

In [ ]:
czind = 0
plt.plot(timevec[indSW[numInd,0]:indSW[numInd,1]],recSig) 
plt.axhline(y=recTr, color='g', linestyle='-.', label='Threshold')
plt.plot(timevec[trueStart[czind]], orgSig[label,trueStart[czind]],'ms',markersize=8, label ='Event marked')                       # Punto inicial cruce FR
plt.plot(timevec[trueEnd[czind]],orgSig[label,trueEnd[czind]],'ms',markersize=8)

plt.xlabel("Time (s)")
plt.ylabel("Amplitude (V)")
plt.title('Rectified Signal')
plt.legend(prop = { "size": 8 } , loc ="upper right")
plt.xlim([timevec[trueStart[czind]]-0.01,timevec[trueEnd[czind]]+0.01])